# Installing and importing Python packages in `pyolite`

`pyolite`, the default Python kernel for JupyterLite powered by
[pyodide](https://pyodide.org), supports installing packages from a number of sources:

- packages distributed with `pyodide`
  - these do not need to be explicitly installed
- packages and shims distributed along with `pyolite`
  - this includes some compatibility shims for the Jupyter stack
- custom packages deployed with a JupyterLite site
- pure python wheel packages on PyPI
  - ending in _exactly_ `py3-none-any.whl`
- or specially compiled wheels for the _exactly_ the version of pyodide and emscripten
  installed
  - ending in e.g. `cp310-cp310-emscripten_3_1_14_wasm32.whl`

## Recommended: `%pip install -q`

The **recommended** way to install packages in pyolite is the `%pip` magic:

In [ ]:
%pip install -q "traitlets >=5" ipython

```{hint}
- `%pip` helps keep your notebooks **portable** between different IPython runtimes.
- `-q` helps keep this "setup" step reasonable when run under `ipykernel`
```

`%pip` supports a number of additional options:

In [ ]:
%pip --help

### `--requirements`

In addition to (or instead of) package names, any number of `--requirements` (or
shorthand `-r`) may be given, pointing to [requirements files][reqs-txt].

[reqs-txt]: https://pip.pypa.io/en/stable/reference/requirements-file-format

These can be relative or absolute (though this can be tricky to predict).

In [ ]:
%pip install -r data/requirements.txt -r ../data/more-requirements.txt

```{hint}
Using a `requirements.txt` is a good way to separate your content from your environment,
and could be combined with other techniques to make even more portable notebooks.
```

#### `requirement.txt` lines

A number of simple types of requirement specifications are
[supported](./data/requirements.txt), including:

- package names
- package names with version specifiers supported by `micropip`
- URLs of `.whl` archives
- `-r` to _more_ requirements files
  - relative to the _requiring_ file, not the `%pip` working directory

Some known **unsupported** features:

- `.` or other local in-development paths
  - but you _could_
    - `%pip install` a build tool (e.g. `flit`)
      - use its Python API
        - install the resulting `.whl`
- `--editable` (or `-e.`) local or remote paths
- any version control system (VCS) paths
- non-`.whl` URLs or local archives
- `--constraint` (or `-c`) contraint files

### Dealing with (missing) dependencies

Even if a PyPI package _would be_ installable on pyolite, sometimes its dependencies
won't be.

In [ ]:
try:
    %pip install jupyter_server
except Exception as err:
    print(err)

### `--verbose`

As the real `pip` doesn't have an equivalent, `%pip` in pyolite maps the `--verbose`
flag to `keep_going`.

In [ ]:
try:
    %pip install --verbose jupyter_server
except Exception as err:
    print(err)

```{warning}
Leaving this on for real `pip` generates a **lot** of output!
```

### `--no-deps`

If some missing dependencies don't bother you, you can forge ahead without _any_
dependencies with the `--no-deps` flag.

In [ ]:
%pip install jupyter_server --no-deps
import jupyter_server

jupyter_server.__version__

While importable, it won't have all of its features, and may require special approaches
to access features.

In [ ]:
try:
    import jupyter_server.services.contents.filemanager
except Exception as err:
    print(err)

Going down this road can be long, depending on how much you really need a particular
function.

## The Hard Way

The `piplite` package is importable, and can be used directly.

### Importing `piplite`

`piplite` needs to be imported before it is used.

In [ ]:
import piplite

This package **won't** be installable in a "traditional" IPython installation, so you
can gate it with an import check:

In [ ]:
try:
    import piplite
except ImportError:
    piplite = None

### `piplite.install`

> `piplite.install` is a wrapper around
> [`micropip.install`](https://pyodide.org/en/stable/usage/loading-packages.html#micropip),
> and offers more browser-focused options than `%pip`

```{warning}
Due to browser limitations, `piplite.install` is an _asynchronous` function, so it must be `await`ed.
```

`piplite.install` supports either a single package:

In [ ]:
await piplite.install("traitlets")

or a list of packages:

In [ ]:
await piplite.install(["traitlets", "IPython"])

It also has many additional options:

In [ ]:
?piplite.install